In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
import requests
import json
import gmaps
import os

# Importing Google maps API key
from api_keys import g_key

prices_path = "Data files/prices with postcode.csv"
salary_path = "Data files/vic_wages_postcode.csv"

prices_df = pd.read_csv(prices_path)
salary_df = pd.read_csv(salary_path)

In [20]:
combined_data = pd.merge(prices_df, salary_df, on="Postcode", how="left")

In [21]:
combined_data

,Suburb,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income
0,ABBOTSFORD,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443
1,ABERFELDIE,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179
2,AIREYS INLET,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581
3,AIRPORT WEST,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710
4,ALBANVALE,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280
...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,YARRAM,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846
661,YARRAVILLE,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410
662,YARRAWONGA,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593
663,YEA,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632


In [22]:
combined_data["Lat"] = ""
combined_data["Lng"] = ""

In [23]:
params = {"key": g_key}

for index, row in combined_data.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    
    suburb = row["Suburb"]
    
    params["address"] = f"{suburb}, Vic"
    
    lat_lng = requests.get(base_url, params=params)
    
    lat_lng = lat_lng.json()
    
    try:
        combined_data.loc[index, "Lat"] = lat_lng["results"][0]["geometry"]["location"]["lat"]
        combined_data.loc[index, "Lng"] = lat_lng["results"][0]["geometry"]["location"]["lng"]
    
    except:
        combined_data.loc[index, "Lat"] = np.nan
        combined_data.loc[index, "Lng"] = np.nan

In [24]:
combined_data

,Suburb,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income,Lat,Lng
0,ABBOTSFORD,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443,-37.803,145.002
1,ABERFELDIE,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179,-37.762,144.901
2,AIREYS INLET,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581,-38.4615,144.106
3,AIRPORT WEST,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710,-37.7262,144.881
4,ALBANVALE,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280,-37.746,144.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,YARRAM,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846,-38.5656,146.675
661,YARRAVILLE,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410,-37.8203,144.882
662,YARRAWONGA,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593,-36.0271,145.999
663,YEA,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632,-37.2229,145.407


In [25]:
combined_data = combined_data.dropna()
combined_data

,Suburb,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income,Lat,Lng
0,ABBOTSFORD,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443,-37.803,145.002
1,ABERFELDIE,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179,-37.762,144.901
2,AIREYS INLET,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581,-38.4615,144.106
3,AIRPORT WEST,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710,-37.7262,144.881
4,ALBANVALE,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280,-37.746,144.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,YARRAM,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846,-38.5656,146.675
661,YARRAVILLE,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410,-37.8203,144.882
662,YARRAWONGA,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593,-36.0271,145.999
663,YEA,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632,-37.2229,145.407


In [28]:
locations = combined_data[["Lat", "Lng"]]

salary = combined_data["Average income"]/1000

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=salary, dissipating=False, max_intensity=10, point_radius=0.05)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))